In [26]:
import pandas as pd

train = pd.read_csv("train.csv")


print(train[0:10])

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   
5   6          50       RL         85.0    14115   Pave   NaN      IR1   
6   7          20       RL         75.0    10084   Pave   NaN      Reg   
7   8          60       RL          NaN    10382   Pave   NaN      IR1   
8   9          50       RM         51.0     6120   Pave   NaN      Reg   
9  10         190       RL         50.0     7420   Pave   NaN      Reg   

  LandContour Utilities  ... PoolArea PoolQC  Fence MiscFeature MiscVal  \
0         Lvl    AllPub  ...        0    NaN    NaN         NaN       0   
1         Lvl    AllPub  ...       

In [27]:
types = train.dtypes


In [28]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

ke = train.columns[2]
print(train[ke])



0       RL
1       RL
2       RL
3       RL
4       RL
        ..
1455    RL
1456    RL
1457    RL
1458    RL
1459    RL
Name: MSZoning, Length: 1460, dtype: object


In [29]:
cat_dict = {}

cat_labels = []
new_arr = []
first = True
non_cat = []

for ke, type in zip(train.columns, types):
    if type == "object":
        le = OneHotEncoder()
        if train[ke].isnull().values.any():
            ifnan = True
        else:
            ifnan = False

        if ifnan:
            unique = np.expand_dims(np.asarray(["nan"] + list(train[ke].dropna().unique())), axis = 1)
        else:
            unique = np.expand_dims(np.asarray(list(train[ke].dropna().unique())), axis = 1)


        train[ke] = train[ke].fillna("nan")

        cat = le.fit_transform(np.expand_dims(train[ke], axis = 1)).toarray().T
        if first:
            first = False
            categorical= cat
        else:
            categorical = np.concatenate((categorical, cat), axis = 0)

        keys = le.categories_
        ka = [k for k in keys]
        cat_dict[ke] = ka
        cat_labels.extend(ka)
    else:
        non_cat.append(ke)

    


In [30]:
categorical = categorical.T



In [31]:
train_no_lab = train.iloc[:,1:-1]
train_lab = train.iloc[:,-1]

train_non_cat = train_no_lab[[t for t in train_no_lab.columns if t in non_cat]]

train_arr = np.concatenate((train_non_cat, categorical), axis = 1)

print(train_arr.shape)





(1460, 304)


In [32]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors = 5)

imputer.fit(train_arr)
train_arr = imputer.transform(train_arr)
print(train_arr.shape)

(1460, 304)


In [33]:
##### Try Linear Model
from sklearn.linear_model import LinearRegression
reg = LinearRegression(fit_intercept = False)
reg.fit(train_arr, train_lab)

LinearRegression(fit_intercept=False)

In [34]:
from sklearn.model_selection import cross_val_score
train_pred = reg.predict(train_arr)
train_score = reg.score(train_arr, train_lab)
reg_scores = cross_val_score(reg, train_arr, train_lab, cv = 5)
print(reg_scores)
print(reg_scores.mean())

[0.81879563 0.82198587 0.79982351 0.88558431 0.67080625]
0.7993991142625093


In [35]:
#### Trying LASSO
from sklearn.linear_model import Lasso
clf = Lasso(alpha=0.1)
clf.fit(train_arr, train_lab)
train_pred_lasso = clf.predict(train_arr)
train_score_lasso = clf.score(train_arr, train_lab)


In [36]:

scores = cross_val_score(clf, train_arr, train_lab, cv = 5)
print(scores)
print(scores.mean())

[0.85706054 0.81845762 0.81089544 0.88835349 0.65434725]
0.8058228689443153


In [38]:
### hyperparameter tuning
alphas = [s/10 for s in range(49,100)]
cv_score = []
for alp in range(49,100):
    alp = alp/10
    clf = Lasso(alpha=alp)
    clf.fit(train_arr, train_lab)
    train_pred_lasso = clf.predict(train_arr)
    scores = cross_val_score(clf, train_arr, train_lab, cv = 5)
    cv_score.append(scores.mean())

print(cv_score)


[0.810443551581948, 0.8105265649161406, 0.8106103849874724, 0.8106929912353728, 0.8107732327534773, 0.8108595302728286, 0.8109472437801497, 0.811034044989386, 0.8111185095037449, 0.811202494325444, 0.8112860215374951, 0.8113722212797583, 0.8114581698113925, 0.811541844023466, 0.8116236406700221, 0.8116992010972505, 0.8117729367532001, 0.8118618879826119, 0.8119872259355191, 0.8121109995468231, 0.8122307561203982, 0.8123495086937469, 0.8124666890859281, 0.8125837289095404, 0.8127008318408449, 0.8128148000767507, 0.8129277746199147, 0.8130401825339362, 0.8131515980039807, 0.8132624575826147, 0.813371378211078, 0.813475010687769, 0.8135772860192478, 0.8136789278558261, 0.8137798481411285, 0.813880355045856, 0.8139801916874825, 0.8140791854254701, 0.8141827223216419, 0.8142943383082575, 0.8144065477058178, 0.8145176076500494, 0.8146268165369183, 0.8147354028961328, 0.8148436787561231, 0.8149533593862316, 0.8150628762666472, 0.8151708194554518, 0.8152796179220443, 0.8153876412349363, 0.8154

In [41]:
max_val = max(cv_score)
indices_max = [i for i,v in enumerate(cv_score) if v == max_val]
print(max_val)
max_alphas = [v for i,v in enumerate(alphas) if i in indices_max]
print(max_alphas)


0.8154958026779349
[9.9]


In [42]:
from sklearn.linear_model import ElasticNet
regr = ElasticNet(random_state=0)
regr.fit(train_arr, train_lab)
el_scores = cross_val_score(regr, train_arr, train_lab, cv = 5)
print(el_scores)



[0.86628579 0.8181168  0.848929   0.84852352 0.64645998]


In [43]:
alphas_el = [s/20 for s in range(1,20)]
cv_score_el = []
for alp in range(1,20):
    alp = alp/20
    regr = ElasticNet(alpha = alp, random_state=0)
    regr.fit(train_arr, train_lab)
    scores = cross_val_score(regr, train_arr, train_lab, cv = 5)
    cv_score_el.append(scores.mean())

KeyboardInterrupt: 

In [66]:
max_val = max(cv_score_el)
indices_max = [i for i,v in enumerate(cv_score_el) if v == max_val]
print(max_val)
max_alphas_el = [v for i,v in enumerate(alphas_el) if i in indices_max]
print(max_alphas_el)

0.8390617568482248
[0.05]


## Regression Trees

In [37]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
random_forest = RandomForestRegressor(max_depth = 10, random_state = 0)
random_forest.fit(train_arr, train_lab)
scores = random_forest.score(train_arr, train_lab)
scores_cv = cross_val_score(random_forest, train_arr, train_lab, cv = 5)


[0.86584411 0.84465632 0.87428482 0.88388224 0.80542397]


In [ ]:
### hyperparameter tuning
alphas_rf = [s for s in range(2,20)]
cv_score_rf = []
for alp in range(2,20):
    regr = RandomForestRegressor(max_depth = alp, random_state = 0)
    regr.fit(train_arr, train_lab)
    scores = cross_val_score(regr, train_arr, train_lab, cv = 5)
    cv_score_rf.append(scores.mean())

In [ ]:
max_val = max(cv_score_rf)
indices_max = [i for i,v in enumerate(cv_score_rf) if v == max_val]
print(max_val)
max_alphas_rf = [v for i,v in enumerate(alphas_rf) if i in indices_max]
print(max_alphas_rf)